<a href="https://colab.research.google.com/github/lut-ful/NLP-healthcare/blob/main/bert-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install transformers datasets accelerate bitsandbytes

In [1]:
!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
# dataset_path="MeDAL-dataset-small-5perc"
# dataset = load_from_disk(dataset_path)
dataset = load_dataset("lutful2004/MeDAL-dataset-small-5perc")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/102M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

data/valid-00000-of-00001.parquet:   0%|          | 0.00/34.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 50000
    })
    valid: Dataset({
        features: ['ABSTRACT_ID', 'TEXT', 'LOCATION', 'LABEL'],
        num_rows: 50000
    })
})


In [3]:
from transformers import AutoTokenizer

# Initialize tokenizer
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

special_tokens = {"additional_special_tokens": ["[ABBR]", "[/ABBR]"]}
tokenizer.add_special_tokens(special_tokens)


def preprocess_function(examples):
    marked_texts = []

    for text, loc, label in zip(
        examples["TEXT"], examples["LOCATION"], examples["LABEL"]
    ):
        # Step 1: Mark abbreviation
        before = text[:loc]
        after = text[loc:]
        tokens = after.split(maxsplit=1)
        abbr = tokens[0]
        rest = tokens[1] if len(tokens) > 1 else ""
        marked_text = f"{before} [ABBR] {abbr} [/ABBR] {rest}"
        marked_texts.append(marked_text)


    # Step 3: Tokenize without tensors
    model_inputs = tokenizer(
        marked_texts,
        padding="max_length",
        truncation=True,
        max_length=128,
        # return_attention_mask=True,
    )

    model_inputs["labels"] = examples["LABEL"]  # Note: Trainer expects "labels" key
    return model_inputs


tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=dataset["train"].column_names
)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_labels = dataset["train"].features["LABEL"].num_classes
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels  # Only pass this
)


# Resize embeddings if tokenizer has extra tokens
model.resize_token_embeddings(len(tokenizer))

# model.gradient_checkpointing_enable()  # save memory


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(30524, 768, padding_idx=0)

In [13]:
print(device)  # should say "cuda"
model.to(device)

cpu


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30524, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
from datasets import Dataset

# Take subset for quick Colab test
train_subset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))
valid_subset = tokenized_dataset["valid"].shuffle(seed=42).select(range(1000))

print(train_subset)
print(valid_subset)


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})


In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/medal-bert-checkpoints",
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=2000,
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=8,     # small batch
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,     # effective batch size = 32
    num_train_epochs=1,
    fp16=True,                         # mixed precision
    logging_dir="./logs",
    # logging_steps=200,
    report_to="none",
    remove_unused_columns=False,
)


In [11]:
from transformers import Trainer, DataCollatorWithPadding
from sklearn.metrics import accuracy_score

data_collator = DataCollatorWithPadding(tokenizer)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=valid_subset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-1398541577.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
metrics = trainer.evaluate(tokenized_dataset["test"]shuffle(seed=42).select(range(1000)))
print(metrics)


In [ ]:
# import torch
# from transformers import (
#     AutoModelForSequenceClassification,
#     BitsAndBytesConfig,
#     TrainingArguments,
#     Trainer,
#     DataCollatorWithPadding,
#     TrainerCallback
# )
# from sklearn.metrics import accuracy_score

# # ===============================
# # 1. Model & Device Setup
# # ===============================
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Load model with quantization and auto device placement
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_checkpoint,
#     num_labels=num_labels,
#     id2label=id2label,
#     label2id=label2id,
#     device_map="auto"
# )

# # Enable gradient checkpointing to save memory
# model.gradient_checkpointing_enable()

# # Optional: split layers manually if needed (multi-GPU)
# # model.layer1.to('cuda:0')
# # model.layer2.to('cuda:1')

# # ===============================
# # 2. Training Arguments
# # ===============================
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="steps",
#     eval_steps=500,
#     save_strategy="steps",
#     save_steps=500,
#     save_total_limit=2,
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,     # small batch
#     per_device_eval_batch_size=4,
#     gradient_accumulation_steps=4,     # effective batch size = 16
#     num_train_epochs=2,
#     fp16=True,                         # mixed precision
#     logging_dir="./logs",
#     logging_steps=100,
#     report_to="none",
#     dataloader_num_workers=0
# )

# # ===============================
# # 3. Data Collator & Metrics
# # ===============================
# data_collator = DataCollatorWithPadding(tokenizer)

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = logits.argmax(axis=-1)
#     return {"accuracy": accuracy_score(labels, predictions)}

# # ===============================
# # 4. Trainer Callback to Clear CUDA Cache
# # ===============================
# class ClearCudaCacheCallback(TrainerCallback):
#     def on_epoch_end(self, args, state, control, **kwargs):
#         torch.cuda.empty_cache()
#         return control

# # ===============================
# # 5. Optimized Trainer
# # ===============================
# class MemoryOptimizedTrainer(Trainer):
#     def evaluation_step(self, model, inputs):
#         """Override evaluation to use torch.no_grad() to save memory."""
#         model.eval()
#         with torch.no_grad():
#             outputs = model(**inputs)
#         return outputs

# trainer = MemoryOptimizedTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["valid"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

# trainer.add_callback(ClearCudaCacheCallback)

# # ===============================
# # 6. Start Training
# # ===============================
# trainer.train()
